In [138]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import matplotlib as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split #to create model and validation data splits
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.model_selection import cross_val_score #to perform k fold cross validation

from sklearn.metrics import accuracy_score,precision_score,recall_score, classification_report, confusion_matrix,f1_score, fbeta_score, roc_auc_score,roc_curve

Loading Dataset

In [9]:
default_data = pd.read_csv('default_data.csv')
manufacturer_data = pd.read_csv('manufacturer.csv')
branch = pd.read_csv('branch.csv')
state = pd.read_csv('state.csv')
default_data.head() #loading all the datasets

,UniqueID,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Date.of.Birth,Employment.Type,DisbursalDate,State_ID,Employee_code_ID,MobileNo_Avl_Flag,Aadhar_flag,PAN_flag,VoterID_flag,Driving_flag,Passport_flag,PERFORM_CNS.SCORE,PERFORM_CNS.SCORE.DESCRIPTION,PRI.NO.OF.ACCTS,PRI.ACTIVE.ACCTS,PRI.OVERDUE.ACCTS,PRI.CURRENT.BALANCE,PRI.SANCTIONED.AMOUNT,PRI.DISBURSED.AMOUNT,SEC.NO.OF.ACCTS,SEC.ACTIVE.ACCTS,SEC.OVERDUE.ACCTS,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES,loan_default
0,420825,50578,58400,89.55,67,22807,45,1441,01-01-1984,Salaried,03-08-2018,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
1,537409,47145,65550,73.23,67,22807,45,1502,31-07-1985,Self employed,26-09-2018,6,1998,1,1,0,0,0,0,598,Medium Risk,1,1,1,27600,50200,50200,0,0,0,0,0,0,1991,0,0,1,1yrs 11mon,1yrs 11mon,0,1
2,417566,53278,61360,89.63,67,22807,45,1497,24-08-1985,Self employed,01-08-2018,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
3,624493,57513,66113,88.48,67,22807,45,1501,30-12-1993,Self employed,26-10-2018,6,1998,1,1,0,0,0,0,305,Very High Risk,3,0,0,0,0,0,0,0,0,0,0,0,31,0,0,0,0yrs 8mon,1yrs 3mon,1,1
4,539055,52378,60300,88.39,67,22807,45,1495,09-12-1977,Self employed,26-09-2018,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,1,1


In [23]:
state.head()

,state_id,event_state
0,6,6058
1,4,7387
2,3,7311
3,9,2036
4,5,1335


In [25]:
state.rename(columns = {'state_id' : 'State_ID'}, inplace = True) #correcting the column name so it matches as master data

Merging datasets

In [27]:
df_1 = pd.merge(default_data, manufacturer_data, on = 'manufacturer_id', how = 'left') #joining manufacturer data
df_2 = pd.merge(df_1, branch, on = 'branch_id', how = 'left') #joining branch data
df = pd.merge(df_2, state, on = 'State_ID', how = 'left') # joining state data
df.head()

,UniqueID,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Date.of.Birth,Employment.Type,DisbursalDate,State_ID,Employee_code_ID,MobileNo_Avl_Flag,Aadhar_flag,PAN_flag,VoterID_flag,Driving_flag,Passport_flag,PERFORM_CNS.SCORE,PERFORM_CNS.SCORE.DESCRIPTION,PRI.NO.OF.ACCTS,PRI.ACTIVE.ACCTS,PRI.OVERDUE.ACCTS,PRI.CURRENT.BALANCE,PRI.SANCTIONED.AMOUNT,PRI.DISBURSED.AMOUNT,SEC.NO.OF.ACCTS,SEC.ACTIVE.ACCTS,SEC.OVERDUE.ACCTS,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES,loan_default,event_count_manufacturer,event_count_branch,event_state
0,420825,50578,58400,89.55,67,22807,45,1441,01-01-1984,Salaried,03-08-2018,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0,12149,2430,6058
1,537409,47145,65550,73.23,67,22807,45,1502,31-07-1985,Self employed,26-09-2018,6,1998,1,1,0,0,0,0,598,Medium Risk,1,1,1,27600,50200,50200,0,0,0,0,0,0,1991,0,0,1,1yrs 11mon,1yrs 11mon,0,1,12149,2430,6058
2,417566,53278,61360,89.63,67,22807,45,1497,24-08-1985,Self employed,01-08-2018,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0,12149,2430,6058
3,624493,57513,66113,88.48,67,22807,45,1501,30-12-1993,Self employed,26-10-2018,6,1998,1,1,0,0,0,0,305,Very High Risk,3,0,0,0,0,0,0,0,0,0,0,0,31,0,0,0,0yrs 8mon,1yrs 3mon,1,1,12149,2430,6058
4,539055,52378,60300,88.39,67,22807,45,1495,09-12-1977,Self employed,26-09-2018,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,1,1,12149,2430,6058


In [95]:
df.head()

,UniqueID,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Date.of.Birth,Employment.Type,DisbursalDate,State_ID,Employee_code_ID,MobileNo_Avl_Flag,Aadhar_flag,PAN_flag,VoterID_flag,Driving_flag,Passport_flag,PERFORM_CNS.SCORE,PERFORM_CNS.SCORE.DESCRIPTION,PRI.NO.OF.ACCTS,PRI.ACTIVE.ACCTS,PRI.OVERDUE.ACCTS,PRI.CURRENT.BALANCE,PRI.SANCTIONED.AMOUNT,PRI.DISBURSED.AMOUNT,SEC.NO.OF.ACCTS,SEC.ACTIVE.ACCTS,SEC.OVERDUE.ACCTS,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES,loan_default,event_count_manufacturer,event_count_branch,event_state
0,420825,50578,58400,89.55,67,22807,45,1441,01-01-1984,Salaried,03-08-2018,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0,12149,2430,6058
1,537409,47145,65550,73.23,67,22807,45,1502,31-07-1985,Self employed,26-09-2018,6,1998,1,1,0,0,0,0,598,Medium Risk,1,1,1,27600,50200,50200,0,0,0,0,0,0,1991,0,0,1,1yrs 11mon,1yrs 11mon,0,1,12149,2430,6058
2,417566,53278,61360,89.63,67,22807,45,1497,24-08-1985,Self employed,01-08-2018,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0,12149,2430,6058
3,624493,57513,66113,88.48,67,22807,45,1501,30-12-1993,Self employed,26-10-2018,6,1998,1,1,0,0,0,0,305,Very High Risk,3,0,0,0,0,0,0,0,0,0,0,0,31,0,0,0,0yrs 8mon,1yrs 3mon,1,1,12149,2430,6058
4,539055,52378,60300,88.39,67,22807,45,1495,09-12-1977,Self employed,26-09-2018,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,1,1,12149,2430,6058


In [112]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233154 entries, 0 to 233153
Data columns (total 44 columns):
 #   Column                               Non-Null Count   Dtype         
---  ------                               --------------   -----         
 0   UniqueID                             233154 non-null  int64         
 1   disbursed_amount                     233154 non-null  int64         
 2   asset_cost                           233154 non-null  int64         
 3   ltv                                  233154 non-null  float64       
 4   branch_id                            233154 non-null  int64         
 5   supplier_id                          233154 non-null  int64         
 6   manufacturer_id                      233154 non-null  int64         
 7   Current_pincode_ID                   233154 non-null  int64         
 8   Date.of.Birth                        233154 non-null  object        
 9   Employment.Type                      233154 non-null  object        
 

In [47]:
df.isnull().sum() #checking total null value

UniqueID                               0
disbursed_amount                       0
asset_cost                             0
ltv                                    0
branch_id                              0
supplier_id                            0
manufacturer_id                        0
Current_pincode_ID                     0
Date.of.Birth                          0
Employment.Type                        0
DisbursalDate                          0
State_ID                               0
Employee_code_ID                       0
MobileNo_Avl_Flag                      0
Aadhar_flag                            0
PAN_flag                               0
VoterID_flag                           0
Driving_flag                           0
Passport_flag                          0
PERFORM_CNS.SCORE                      0
PERFORM_CNS.SCORE.DESCRIPTION          0
PRI.NO.OF.ACCTS                        0
PRI.ACTIVE.ACCTS                       0
PRI.OVERDUE.ACCTS                      0
PRI.CURRENT.BALA

In [43]:
df['Employment.Type'].fillna('unknown', inplace= ) #replacing null values of Employment.Type with 'unknown'

/var/folders/fr/p9_lmgfd7w576k_9rfjc94j00000gn/T/ipykernel_53906/2910275874.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Employment.Type'].fillna('unknown', inplace= True)


In [49]:
df['Employment.Type'].value_counts()

Employment.Type
Self employed    127635
Salaried          97858
unknown            7661
Name: count, dtype: int64

In [97]:
df['AVERAGE.ACCT.AGE'] = df['AVERAGE.ACCT.AGE'].str.replace('yrs', '*12').str.replace(' ', '+').str.replace('mon', '*1').apply(eval)
#converting expression of 0yrs 0mon to total months using string replace function 

In [99]:
df['CREDIT.HISTORY.LENGTH'] = df['CREDIT.HISTORY.LENGTH'].str.replace('yrs', '*12').str.replace(' ', '+').str.replace('mon', '*1').apply(eval)
# same operation as AVERAGE.ACCT.AGE on CREDIT.HISTORY.LENGTH too

In [110]:
df['DisbursalDate'] = pd.to_datetime(df['DisbursalDate'], format = "%d-%m-%Y") #changing disbursal date format to datetime from object.

In [118]:
df['days_since_disbursal'] = (pd.to_datetime('today') - df['DisbursalDate']).dt.days #creating new variable days_since_disbursal

In [126]:
df = pd.get_dummies(df, columns=['Employment.Type']) #creating dummies for employement type column

In [128]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233154 entries, 0 to 233153
Data columns (total 47 columns):
 #   Column                               Non-Null Count   Dtype         
---  ------                               --------------   -----         
 0   UniqueID                             233154 non-null  int64         
 1   disbursed_amount                     233154 non-null  int64         
 2   asset_cost                           233154 non-null  int64         
 3   ltv                                  233154 non-null  float64       
 4   branch_id                            233154 non-null  int64         
 5   supplier_id                          233154 non-null  int64         
 6   manufacturer_id                      233154 non-null  int64         
 7   Current_pincode_ID                   233154 non-null  int64         
 8   Date.of.Birth                        233154 non-null  object        
 9   DisbursalDate                        233154 non-null  datetime64[ns]
 

In [174]:
x_train = df[[
    'disbursed_amount', 'asset_cost', 'ltv',
    'event_count_manufacturer', 'event_count_branch', 'event_state',
    'days_since_disbursal',
    'PERFORM_CNS.SCORE',
    'PRI.NO.OF.ACCTS', 'PRI.ACTIVE.ACCTS', 'PRI.OVERDUE.ACCTS',
    'PRI.CURRENT.BALANCE', 'PRI.SANCTIONED.AMOUNT', 'PRI.DISBURSED.AMOUNT',
    'SEC.NO.OF.ACCTS', 'SEC.ACTIVE.ACCTS', 'SEC.OVERDUE.ACCTS',
    'SEC.CURRENT.BALANCE', 'SEC.SANCTIONED.AMOUNT', 'SEC.DISBURSED.AMOUNT',
    'PRIMARY.INSTAL.AMT', 'SEC.INSTAL.AMT',
    'NO.OF_INQUIRIES', 'NEW.ACCTS.IN.LAST.SIX.MONTHS', 'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS',
    'CREDIT.HISTORY.LENGTH', 'AVERAGE.ACCT.AGE',
    'Aadhar_flag', 'PAN_flag', 'VoterID_flag', 'Driving_flag', 'Passport_flag',
    'MobileNo_Avl_Flag',
    'Employment.Type_Salaried', 'Employment.Type_Self employed', 'Employment.Type_unknown'
]] #creating train data

In [176]:
x_train

,disbursed_amount,asset_cost,ltv,event_count_manufacturer,event_count_branch,event_state,days_since_disbursal,PERFORM_CNS.SCORE,PRI.NO.OF.ACCTS,PRI.ACTIVE.ACCTS,PRI.OVERDUE.ACCTS,PRI.CURRENT.BALANCE,PRI.SANCTIONED.AMOUNT,PRI.DISBURSED.AMOUNT,SEC.NO.OF.ACCTS,SEC.ACTIVE.ACCTS,SEC.OVERDUE.ACCTS,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NO.OF_INQUIRIES,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,CREDIT.HISTORY.LENGTH,AVERAGE.ACCT.AGE,Aadhar_flag,PAN_flag,VoterID_flag,Driving_flag,Passport_flag,MobileNo_Avl_Flag,Employment.Type_Salaried,Employment.Type_Self employed,Employment.Type_unknown
0,50578,58400,89.55,12149,2430,6058,2441,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,True,False,False
1,47145,65550,73.23,12149,2430,6058,2387,598,1,1,1,27600,50200,50200,0,0,0,0,0,0,1991,0,0,0,1,23,23,1,0,0,0,0,1,False,True,False
2,53278,61360,89.63,12149,2430,6058,2443,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,False,True,False
3,57513,66113,88.48,12149,2430,6058,2357,305,3,0,0,0,0,0,0,0,0,0,0,0,31,0,1,0,0,15,8,1,0,0,0,0,1,False,True,False
4,52378,60300,88.39,12149,2430,6058,2387,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233149,63213,105405,60.72,9786,1023,6058,2357,735,4,3,0,390443,416133,416133,0,0,0,0,0,0,4084,0,0,0,0,39,21,0,0,1,0,0,1,True,False,False
233150,73651,100600,74.95,4919,1023,6058,2360,825,1,0,0,0,0,0,0,0,0,0,0,0,1565,0,0,0,0,6,6,0,0,1,0,0,1,False,True,False
233151,33484,71212,48.45,22987,428,7387,2359,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,True,False,False
233152,34259,73286,49.10,22987,428,7387,2384,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,True,False,False


In [136]:
y_train = df['loan_default'] #defining y train with target variable

In [144]:
y_train.value_counts()

loan_default
0    157976
1     75178
Name: count, dtype: int64

In [178]:
x_train_model, x_validation,y_train_model, y_validation = train_test_split(x_train, y_train,train_size = 0.8) # model and validation data split

In [220]:
model_1 = LogisticRegression() #initializing logistic model

#fitting the model on train data (80% here)
model_1.fit(x_train_model,y_train_model)

y_validation_prediction = model_1.predict(x_validation) # predicting on validation data
y_validation_prediction_prob = model_1.predict_proba(x_validation) #preficting probability on validation data


print("F1 score: ",f1_score(y_validation,y_validation_prediction)) #true, predicted values - most useful metric in general
print("ROC Score: ",roc_auc_score(y_validation,y_validation_prediction_prob[:,1]))

F1 score:  0.0
ROC Score:  0.5301405284805945


In [217]:
model_RF = RandomForestClassifier(class_weight='balanced', random_state=42) # initializing a random forest model
model_RF.fit(x_train_model,y_train_model) #fitting x and y model data

y_validation_prediction = model_RF.predict(x_validation) #predicting on validation data 
y_validation_prediction_prob = model_RF.predict_proba(x_validation) # predicting probability on validation data

print("F1 score: ",f1_score(y_validation,y_validation_prediction))
print("ROC Score: ",roc_auc_score(y_validation,y_validation_prediction_prob[:,1]))

F1 score:  0.421115266916015
ROC Score:  0.7078161287838864


In [219]:
scores = cross_val_score(model_RF, x_train, y_train, cv=5, scoring='f1')  #doing cross validation on random forest model

print("F1 Scores:", scores)
print("Average F1 Score:", scores.mean())

F1 Scores: [0.44862265 0.44891765 0.39222301 0.36356198 0.3434795 ]
Average F1 Score: 0.3993609575923927


In [186]:
scores = cross_val_score(model_RF, x_train, y_train, cv=5, scoring='roc_auc')  # checking roc_auc score through cross validation

print("ROC AUC Scores:", scores)
print("Average ROC AUC Score:", scores.mean())

ROC AUC Scores: [0.70930896 0.70990848 0.69702088 0.6844331  0.6848559 ]
Average ROC AUC Score: 0.6971054633950393


In [188]:
from sklearn.model_selection import GridSearchCV #hyperparameter tuning and grid search to improve model further

params = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
} # parameters for tuning (Acknowledgement: took internet help in tuning parameters.)

model = RandomForestClassifier(class_weight='balanced', random_state=42) #stating the model
grid = GridSearchCV(model, params, cv=3, scoring='f1') #laying grid search framework
grid.fit(x_train, y_train) #fitting whole x train and y train data

print("Best Params:", grid.best_params_)
print("Best F1 Score:", grid.best_score_)

Best Params: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}
Best F1 Score: 0.5346995160654121


In [190]:
grid.best_estimator_ # best model after fine tuning

RandomForestClassifier(class_weight='balanced', max_depth=10,
                       min_samples_split=5, n_estimators=200, random_state=42)

In [194]:
#checking roc_auc score of same model
roc_scores = cross_val_score(
    grid.best_estimator_,
    x_train, y_train,
    cv=5,
    scoring='roc_auc'
)

print("ROC AUC Scores (CV):", roc_scores)
print("Average ROC AUC Score:", roc_scores.mean())

ROC AUC Scores (CV): [0.71786731 0.72665644 0.71283991 0.69382851 0.68643175]
Average ROC AUC Score: 0.7075247849299147


In [ ]:
best_model = grid.best_estimator_